In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from transformers import BertTokenizer, AutoModelForTokenClassification
import torch
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if filename == 'test_predictions.txt':
            print(os.path.join(dirname, filename))
        if filename == 'cleaned_test.csv':
            print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# # Import model, labels, test dataset
Model is trained using:
https://www.kaggle.com/numerator/run-ner-indobert-base-p1/ version 3

In [ ]:
model = AutoModelForTokenClassification.from_pretrained("/kaggle/input/run-ner-indobert-base-p1/indobert-ner/")
tokenizer = BertTokenizer.from_pretrained('/kaggle/input/run-ner-indobert-base-p1/indobert-ner/')

In [ ]:
label_list = ["128", "O", "POI", "ST"]

In [ ]:
#test_prediction_path = '/kaggle/input/run-ner-indobert-base-p1/indobert-ner/test_predictions.txt'
test_df = pd.read_csv('/kaggle/input/scl2021data/cleaned_test.csv').fillna('')
test_df.head()

In [ ]:
def return_pred(sequence):
    # Bit of a hack to get the tokens with the special tokens
    tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
    inputs = tokenizer.encode(sequence, return_tensors="pt")

    outputs = model(inputs)[0]
    predictions = torch.argmax(outputs, dim=2)

    pred = [(token, label_list[prediction]) for token, prediction in zip(tokens, predictions[0].tolist())]
    pred_df = pd.DataFrame(pred)
    poi = ''
    st = ''
    for i in range(len(pred_df)):
        if pred_df.iloc[i,1] == 'POI':
            poi = poi + ' ' + pred_df.iloc[i,0]
        if pred_df.iloc[i,1] == 'ST':
            st = st + ' ' + pred_df.iloc[i,0]
    return (poi + '/' + st).replace(' ##', '')

# Inference on dev set

dev_df = pd.read_csv('/kaggle/input/scl2021data/dev.csv')
dev_df['POI/street'] = dev_df[['POI','street']].fillna('').agg('/'.join, axis=1)
dev_df

%%time
dev_df['pred'] = dev_df['raw_address'].apply(lambda x: return_pred(x))
dev_df.head()

dev_df['result'] = np.where(df['raw_address'] == df['pred'], 1, 0)
print(dev_df['result'].sum() / dev_df['result'].count())

# Inference on test set

In [ ]:
%%time
test_df['POI/street'] = test_df['raw_address'].apply(lambda x: return_pred(x))
test_df.to_csv('/kaggle/working/submit.csv', index=False)
test_df.head()

def preprocess_test_data(raw_df, output_name):
    start_counter = 1
    end_counter = 0
    raw_df['start'] = start_counter
    raw_df['end'] = end_counter
    with open(output_name, 'a') as text_file:
        for index, row in tqdm(raw_df.iterrows()):
            raw_df.loc[index, 'start'] = start_counter
            address = row['raw_address'].replace(",", "").split()
            for address_word in address:
                text_file.write(address_word + '\n')
                start_counter += 1
                end_counter += 1
            raw_df.loc[index, 'end'] = end_counter
    return raw_df

test_processed_df = preprocess_test_data(test_df, 'test_temp.txt')
test_processed_df.head()

def extract_word(file, row_start, row_end):
    '''
    format model prediction output in submission format
    '''
    poi = ''
    st = ''
    f=open(file)
    lines = f.readlines()
    i = row_start
    while i >= row_start and i <= row_end:
        if len(lines[i]) > 1 :
            word = lines[i].split()[0]  
            tag = lines[i].split()[1]
        else:
            tag = 'O'
        if tag == 'POI':
            poi = poi + ' ' + word
        elif tag == 'ST':
            st = st + ' ' + word
        i += 1
    return poi.strip() + '/' + st.strip()

test_processed_df['POI/street']=''
printcounter = 0
for i in tqdm(range(0, len(test_processed_df))):
    row_start = test_processed_df.loc[i, 'start']
    row_end = test_processed_df.loc[i, 'end']
    test_processed_df.loc[i, 'POI/street'] = extract_word(test_prediction_path, row_start, row_end)
    # add checkpoints
    if (printcounter == 1000):
        test_processed_df[['id','POI/street']].to_csv('submit.csv', index=False)
        printcounter = 0
        printcounter += 1
        
test_processed_df[['id','POI/street']].to_csv('submit.csv', index=False)
